# Query-Document Relevence Ranking model

검색 서비스 만족도 판별 모델에 이어 검색 서비스 랭킹 모델을 개발한 과정을 정리해 봤습니다. 먼저 참고한 논문들을 소개하고 구현해보겠습니다.

## A Deep Relevance Matching Model for Ad-hoc Retrieval (DRMM)

### DRMM introduction

검색 서비스를 제공할때 고려하여야 할점은 검색어와 검색 결과의 연관성뿐만 아니라 연관성이 높은 결과를 상위에 노출시키는것 또한 중요합니다. 딥러닝은 여러 분야에서 SOTA를 달성하고 있습니다. DRMM이 제안되기 전 딥러닝을 통한 검색 서비스 랭킹 시스템은 전통적인 머신러닝 방식을 훨신 뛰어넘는 성과를 달성하지 못하였습니다.

랭킹 시스템에서 일반적인 딥러닝 모델은 두 text의 매칭문제로 바라보았습니다. 이에 논문은 모델 접근법에 따라 **표현중심(representation-forcused) 모델**(DSSM, ARC-I)과 **상호작용중심(Interaction-focused) 모델**(Deep Match, ARC-II)로 구분하고 이에 따른 모델 아키텍쳐를 설명합니다.

표현중심 모델은 각 text의 **의미**를 잘 표현하는 구조에 집중하여 검색어와 검색결과의 의미를 매칭시키는데 집중합니다. 상호작용 중심 모델은 두 text의 상호작용을 계층적으로 배워 **관련성**을 매칭시키는데 집중합니다. 논문은 딥러닝이 좋은 성과를 내지못한 이유가 제안되었던 접근 방식들이, 의미론적 매칭을 위한, 표현중심 모델링이기 때문이라고 주장 하였습니다.

<img src='./img/drmm_fig1.png'>

표현중심 모델(a)은 검색어와 검색결과를 각각 input으로써 모델에 입력하여 대칭적인 아키텍쳐를 보입니다. 반면에, 상호작용중심 모델(b)은 검색어와 검색결과의 연산을 거쳐 계층적인 일치도를 파악하기에 피라미드 모양의 아키텍쳐를 보입니다.

NLP에서 두 텍스트 사이의 일치도를 이용하는 시스템은 여러가지가 있습니다. 고객이 올린 질문에 대한 답변을 찾아주는 QnA모델, 실시간 채팅서비스를 제공하는 chatbot, 정확한 검색 서비스를 제공하는 랭킹모델등등 언뜻 생각하면 비슷한 구조를 가질것 이라는 생각이 들지만 접근법에 따라 상당히 다른 아케텍쳐가 사용됩니다. 논문은 retrival task는 기존의 의미적 매칭 접근법과 근본적으로 다르며 관련성 기준으로 매칭시켜야 한다고 주장합니다.

논문은 이 차이점을 설명하고 관련성 매칭으로 접근해야하는 이유를 설명합니다.

**The major contributions of this paper include**
1. 의미적 매칭과 관련성 매칭의 차이점을 지적하고 모델 아키텍쳐 설계의 차이점 설명
2. 관련성 매칭의 3가지 핵심요인을 따른 새로운 모델 제안
3. 기존 방식의 단점과 제안한 모델(DRMM)의 장점 분석

### Ad-hoc retrieval as a matching problem

관련성 점수를 계산하는 큰틀은 다음 식과 같습니다.

$$match(T_1, T_2) = F(\Phi(T_1), \Phi(T_2))  $$

$\Phi$는 word2vec처럼 text를 vector로 표현하는 함수입니다.
$F$는 계산된 벡터 표현으로부터 점수를 계산하는 함수입니다.

앞서 설명했듯이 접근법에 따라 함수또한 달라지는데, 일반적으로 표현중심모델은 문장의 벡터표현이 중요하기 때문에 상대적으로 $\Phi$부분이 복잡하고, 상호작용 중심 모델의 경우는 두 텍스트 사이의 상호작용을 계층적으로 파악하려 하기에 $F$부분이 복잡합니다.

### semantic matching VS relevance matching

이번에는 NLP text matching task에서 retrieval matching과 semantic matching의 차이점에대 해서 논의 하겟습니다.

paraphrase identification, question answering, automatic conversation와 같은 NLP task에서의 매칭은 주로 **semantic matching**을 고려합니다. 의미론적 관계 추론은 다음 3가지 핵심요인이 있습니다.

1. similarity matching signals
    + 서로 다르게 표현된 문장들이 같은 의미를 가지는지 다른 의미를 가지는지 식별할 수 있어야 합니다.
    + 예를들어, automatic conversation에서 사용자의 대화에서 의미적으로 관련된 적절한 응답을 찾는 것이 목표이며, 사용자들은 같은 의미를 전달하기 위해서 서로 다른 문장을 사용할수 있습니다.  
1. compositional meanings
   + 의미론적 매칭의 텍스트 데이터는 일반적으로 문법 구조를 가진 자연어 문장으로 구성되어 있습니다. 이는 문법 구조를 포착할 수 있는 모델링이 유리하다는것을 의미합니다.  
   + 예를 들어, QnA 시스템에서 대부분의 질문에는 명확한 문법 구조가있어 질문 내용의 의미를 식별하는 데 도움이됩니다.
1. Global matching requirement
    + 의미론적 매칭은 일반적으로 두 텍스트를 전체적인 의미를 유추하기에 글로벌한 접근법이 필요합니다. 
    
관련성 추론은 다음 3가지 핵심 요인이 있습니다.
1. Exact matching signals
    + 두 텍스트의 의미론적 관련성 신호가 중요하긴 하지만 비교하는 텍스트의 정확한 일치신호가 훨신더 강력한 신호로 다루어야 합니다.
    + 예를들어 Fang and Zhai [[7]](https://www.eecis.udel.edu/~hfang/pubs/sigir06-expansion.pdf)는 의미적으로 비슷한 용어를 여러번 일치시키는 것보다 정확히 같은 용어의 신호를 다루는것이 관련성 점수에 기여한다고 하였습니다.
    + 이는 TF-IDF나 BM25와 같은 전통적인 머신러닝방법이 잘 작동하는 이유이기도 합니다.
1. Query term importance
    + 일반적으로 검색어는 키워드 기반입니다 이는 검색어에서 단어가'and'관계이며 상대적인 중요도가 존재한다는 것입니다. 
    + 예를들어, '비트코인 뉴스'라는 검색어는 '비트코인'과 '뉴스'라는 키워드로 이루어져있으며 중요도는 '비트코인' < '뉴스'로 해석할 수 있습니다.  
1. Diverse matching requirement
    + 검색 결과의 문서가 매우 길어질 수 있으며 문서 길이에 대해 서로 다른 가설이있어 다양한 요구 사항이 발생합니다. 
    + _Verbosity Hypothesis_ : 긴 문서는 짧은 문서와 유사하지만 많은 단어를 담고 있다.
        + 이 경우 짧은 문서에 집중된 주제가 있다고 가정하면 관련성 일치가 Global해야 합니다.
    + _Scope Hypothsis_ : 긴 문서는 여러개의 관련없는 짧은 문서로 구성되어 있다.
        + 이 경우 관련성 일치는 문서의 어느 부분에서나 발생할 수 있으며 문서 전체가 쿼리와 관련 될 필요는 없다고 합니다.
    
이를 볼때 기존 방식은 대부분 의미론적 매칭에 가까우며, 검색서비스에서는 관련도 추론이 더욱 적합합니다.

### Deep Relevance Matching Model

이에 논문은 새로운 상호작용중심 모델 **DRMM**을 제안합니다. 다음 그림은 모델 아키텍처입니다.

<img src='./img/drmm_fig2.png'>
<span style="display: block; text-align: center;">Architecture of the DRMM</span>

다음은 모델 아키텍처의 수식입니다. 각 $w_{i}^{(q)}$과 $w_{j}^{(d)}$는 pretrained word2vec으로 표현된 term vector입니다 아래에 각 수식에 대한 설명을 해보겟습니다.

$$q = \{w_{1}^{(q)}, \dots, w_{M}^{(q)} \}, d = \{w_{1}^{(d)}, \dots, w_{N}^{(d)} \}\quad whrer\quad w_{i}^{(q)}, i = 1, \dots, M\quad and\quad  w_{j}^{(d)}, j = 1, \dots, N$$

$$ Z_{i}^{(0)} = h(w_{i}^{(q)}\otimes d),\quad i = 1, \dots, M$$


$$ Z_{i}^{(l)} = tanh(W^{(l)}Z_{i}^{(l-1)} + b^{(l)}),\quad i = 1, \dots, M, l = 1, \dots, L$$


$$ s = \sum_{i=1}^{M}g_iZ_i^{(L)}$$

#### Local Interaction $(w_{i}^{(q)}\otimes d)$

$\otimes$는 interaction operator입니다. 거창해보이지만 pretrained word2vec모델로 표현된 단어들의 cos 유사도입니다...
* pretrained word2vec를 사용하는 이유는 ground truth보다 Large scale unlabeled text collection에서 학습된 표현이 정확도가 높았고 전체 모델의 복잡성을 낮추기 위해서라고 합니다. 

#### Matching Histogram Mapping $h(x)$

기본적으로 NLP task에서 데이터는 가변길이이므로 이를 고정길이로 표현합니다. 논문은 Local Interaction에서 구한 유사도 행렬이 위에 서술한  Diverse matching requirement를 만족하지 못하기 때문에 잠재적으로 유사도 행렬이 적합하지 않을수 있다고 합니다. 그러므로 DRMM은 히스토그램을 채택하여 위치보다 신호의 강도를 그룹화 합니다. 

예를들어 코사인 유사도를 0.5마다 그룹화하면 {\[−1, −0.5), [−0.5, −0), [0, 0.5), [0.5, 1), [1, 1]}라는 5개의 그룹이 나옵니다. 이때 검색어에서 "car"라는 단어와 문서의 "car, rent, truck, bump, injunction, runway"라는 단어들의 유사도를 구했을때 유사도가 (1, 0.2, 0.7, 0.3, −0.1, 0.1)라고 계산되었다면 그룹화한 유사도 히스토그램은 [0, 1, 3, 1, 1]으로 표현됩니다.

+ 여기서 Exact matching signals 요구사항을 만족시키기 위하여 정확한 일치 신호인 [1, 1]그룹을 명시적으로 표현합니다.

계산된 히스토그램을 그대로 사용할 수 있지만 한번의 계산을 거쳐 좀더 좋은 정보로 바꿉니다.
1. Count-based Histogram (CH)
    + 각 구간에 속한 유사도들의 빈도를 히스토그램으로 사용합니다. 그냥 위에 구한 히스토그램을 그대로 사용하는것입니다.
2. Normalized Histogram (NH)
    + Count-based Histogram를 정규화 해서 절대 갯수가 아닌 상대 갯수정보를 사용합니다.
3. LogCount-based Histogram (LCH)
    + Count-based Histogram에 로그함수를 적용하여 범위를 줄여 모델이 곱셈관계를 더 쉽게 배울수 있게 합니다.
    
정리하자면 히스토그램을 채택함으롰 정확한 매칭과 유사한 매칭신호를 명확하게 구분하면서 가변길이 벡터를 패딩없이 고정길이 벡터로 표현이 가능해집니다.

#### Feed forward Matching Network $Z_{i}^{(l)} = tanh(W^{(l)}Z_{i}^{(l-1)} + b^{(l)})$

Matching Network에서는 계층적 일치 패턴을 학습합니다. 계층적 매칭 패턴을 학습하기 위한 기존의 접근 방법은 CNN을 사용함으로써 계층적 패턴을 학습하려 했습니다. 이러한 모델은 (Local)Receptive Field가 있는 Conv unit을 사용하고 일치 패턴에서 위치 규칙을 학습합니다. 즉 Global matching requirement를 만족하므로 의미론적 매칭 모델에서 잘 작동합니다.

하지만 논문은 검색어에서 위치 규칙성이 없을 수 있으므로 CNN은 적합하지 않다고 판단하여 MLP를 채택합니다. 이에 DRMM은 위치보다 강도에 신호의 강도에 중점을 두어 더욱 유리하다고 합니다.

#### Term Gating Network $s = \sum_{i=1}^{M}g_iZ_i^{(L)}$

검색어의 중요성을 명시적으로 모델링하는 부분입니다. 각 $g_i$들은 아래의 식으로 계산됩니다.

$$ g_i = \frac{exp(w_gx_i^{(q)})}{\sum_{j=1}^{M}exp(w_gx_j^{(q)})}$$

여기서 $w_g$는 Term Gating Network Weight이며 $x_i^{(q)}$는 수치로 표현된 검색어의 각 단어입니다.

논문은 단어의 수치적 표현을 두가지 방법으로 시도했습니다.
1. Term Vector(TV)
    + 임베딩된 단어를 그대로 사용합니다. 여기서 가중치의 차원또한 임베딩 차원과 같게 맞춰줍니다.
2. Inverse Document Frequency(IDF)
    + NLP task에서 많이 사용하는 IDF를 채택함으로써 각 단어의 중요성 정보또한 모델에 입력합니다. 여기서 가중치는 sigle parameter가 됩니다.

#### Model Training

모델을 학습하기 위한 Loss Function으로는 Pairwise ranking loss를 채택합니다.

$$L(q, d^+, d^-;\theta) = max(0, 1 - s(q, d^+) + s(q, d^-))$$

여기서 $d^+$는 $q$에 대해 관련성이 높은 문서입니다. 모델 학습과정에서는 SGD, mini-batch(20 in size), regularuzation, early stopping을 사용합니다. 전처리는 HTML tag 제거, stemming, word2vec(min_df=10, 300dims, CBOW, window_size=10)을 사용합니다.

### Model Implement

논문에서 소개한 모델 평가는 검색 서비스 만족도 판별 모델에서 사용하였던 Crowdflower 데이터를 사용하여 모델을 구현한뒤 직접 비교해보도록 하겟습니다.

자세한 code는 github을 참고해 주세요

```python
class DRMM(tf.keras.Model):
    def __init__(self):
        super(DRMM, self).__init__(name='DRMM')
        self._supports_ragged_inputs = True
        self.Word_Matching_Network = Word_Matching_Network()
        self.Gating_Network = Gating_Network()
        self.Score_Aggregation = Score_Aggregation()

    def call(self, inputs, idf):
        Z = self.Word_Matching_Network(inputs)
        G = self.Gating_Network(idf)
        score = self.Score_Aggregation(Z, G)
        return score
```

#### Evaluation

평가는 [nDCG@20](https://en.wikipedia.org/wiki/Discounted_cumulative_gain)으로 해보겟습니다.(1에 가까울수록 좋은 지표입니다.) 

<img src='./img/drmm_loss.PNG'>

사실 직접 구현하는것이 아닌이상 단순히 평가 지표로는 얼마나 좋은 모델이지 감이 오지 않습니다. 또한 가지고 있는 데이터가 관련도가 4인 data가 60%가까이 되므로 nDCG@20값이 높게 계산됩니다.  dev set에서 검색어를 랜덤하게 뽑아 랭킹이 얼마나 잘되었는지 보겠습니다.
먼저 훈련하기전 nike flip flops라는 검색어에서 20개를 랜덤하게 뽑아온 결과입니다. 

<img src='./img/query_before_training.PNG'>

모델 훈련을 마친뒤 랭킹입니다.


<img src='./img/drmm_after_training.PNG'>


## PACRR: A Position-Aware Neural IR Model for Relevance Matching (PACRR)

DRMM에 이어서 나온 관련성 기반 모델입니다. 이론은 DRMM으로 충분하며 이론보다는 모델을 소개하는 논문이므로 바로 Architecture만 간략히 소개하겟습니다.

### The PACRR Model

<img src='./img/pacrr_fig1.png'>
<span style="display: block; text-align: center;">Architecture of the PACRR</span>

PACRR은 두개의 메인 파트로 이루어져 있습니다.
1. Relevance Matching
    + 검색어와 검색결과의 유사도를 계산한 행렬을 처리하는 파트 입니다.
2. Deep Retrieval Model
    + 위에서 구한 유사도 행렬을 입력으로 받아 관련성 점수를 계산하는 파트 입니다.

### Relevance Matching

PACRR도 DRMM과 마찬가지로 pretrained word2vec을 사용하여 단어를 임베딩 합니다. 또한 Local Interaction에서와 같이 검색어-검색결과 쌍의 유사도행렬을 구합니다 다른점은 히스토그램을 사용하지 않고 그림에서와 같이 구해진 유사도 행렬을 $sim_{ \left\vert q \right\vert \times  \left\vert d \right\vert} \rightarrow sim_{l_q \times l_d}$로 고정된 차원의 유사도 행렬로 처리합니다.

여기서 $l_q$는 검색어중 가장 긴 문장의 길이 입니다. 짧은 문장의 검색어는 zero padding으로 차원을 맞춰줍니다.
$l_d$는 하이퍼 파라미터 입니다. $l_q$와 마찬가지로 $l_d$보다 짧은 문장은 zero padding을 거치고 $l_d$보다 긴 문장은 두가지 방법중 하나를 선택하여 처리 합니다.
1. PACRR-firstk
    + 간단하게 앞에서 $l_d$개만 유지하고 뒷부분은 자릅니다.
2. PACRR-kwindow
    + 기본적으로 유사도가 낮은 단어들은 점수에 영향력이 없으므로 관련성이 높은 $l_d$개를 유지합니다.

### Deep Retrieval Model

Relevance Matching에서 구한 행렬을 입력으로 받아서 관련성 점수를 계산하는 end-to-end deep learning model 파트 입니다.

#### Convolutional relevance matching over local text snippets

유사도 행렬에 각각 다른 size의 커널을 이용하여 conv 연산을 합니다. 커널의 사이즈는 $2 \times 2$ 부터 $l_g \times l_g$까지 $l_g - 1$개의 conv layer를 사용합니다 여기서 $l_g$는 하이퍼 파라미터 입니다. 각 커널은 $n_f$개의 필터를 가지고 있으며 output은 $C^2_{l_q \times l_d \times nf} \dots C^{lg}_{l_q \times l_d \times nf}$입니다. 추가로 conv layer를 거치지 않은 유사도 행렬도 하나의 층으로 이용됩니다.

+ 재미있는점은 $n \times n$ conv layer를 적용한다는것은 각 단어들을 n-gram으로 바라본다는 의미로 해석됩니다.

#### Two max pooling layers

Conv layer를 거친후 2단계의 max pooling layer를 통과 합니다. 


1. max pooling over filter dims $n_f$
    + 필터기준 max-pooling을 실행함으로써 각 n-gram에서 연산된 가장 강력한 신호를 추출 합니다.
    + pooling layer를 통과한 output은 $C^1_{l_q \times l_d \times 1} \dots C^{lg}_{l_q \times l_d \times 1}$으로 $l_g$개의 feature map입니다.
    
    
2. ror wise k-max pooling
    + $l_g$개의 feature map에서 row기준 가장 큰 $n_s$개를 남김으로써 각 n-gram에서 가장 강력한 단어쌍의 신호를 추출합니다.
    + 또한 연속적이지 않지만 강력한 신호를 추출하는 효과도 있습니다.
    + pooling layer를 통과한 output은 $P_{l_q \times l_g \times n_s }$ 으로 $l_g$개의 feature map입니다.
    
2단계의 max pooling layer를 통과한 feature map들은 하나의 feature map으로 Concatenation 시킵니다. 

#### softmax IDF stack

계산된 feature map은 row가 검색어의 각 단어들을 의미하고 columns는 각 n-gram에서 $n_s$개의 가장 강력한 신호들 입니다.
다음은 DRMM에서와 같이 IDF값을 계산하여 feature map 마지막 columns에 stack시킵니다. 다른점은 softmax를 사용하여 Normalized효과와 함께 $l_q$보다 짧은 단어에서 zero padding으로 생성된 부분도 중요성을 나누어 줄수 있습니다.

#### Recurrent layer for global relevance

마지막으로 생성된 행렬을 LSTM Layer의 input으로 하여 최종 관련성 점수를 계산합니다. 입력은 $q_i \times P_{l_g \times n_s }$으로 검색어의 단어를 나타내는 row기준으로 나누어 각 cell에 순서대로 입력됩니다.



간단히 정리하자면 검색어와 검색결과의 유사도 행렬을 n-gram으로 처리하면서 가장 강력한 신호 $n_s$개와 $IDF$를 이용하여 관련성을 계산하는것 입니다. DRMM에서 제시한 관련성 추론을 위한 3가지 핵심 요인이 잘 녹아 들어있는지 비교해 보겟습니다.
1. Exact matching signals
    + 2단계의 max pooling을 거치면서 최대한 강력한 신호를 파악하려 하지만 정확한 일치신호를 명시적으로 파악하는 구간은 없습니다. 
    
    
2. Query term importance
    + DRMM에서와 비슷하게 IDF를 사용하여 어느정도 수용될것으로 보입니다.
    
    
3. Diverse matching requirement
    + 문서의 길이는 두가지 하이퍼 파라미터 $l_q, l_d$로 처리하고 있습니다. 
    + PACRR-kwindow처리 방법이 조금더 Global하며 연관성이 높은 쌍을 처리하기 때문에 Verbosity Hypothesis, Scope Hypothsis 둘다 어느정도 만족할수 있다고 생각됩니다.

#### training objective

DRMM과 마찬가지로 Pairwise ranking loss를 채택합니다.

$$L(q, d^+, d^-;\theta) = max(0, 1 - rel(q, d^+) + rel(q, d^-))$$

### Model Implement

```python
class PACRR(tf.keras.Model):
    def __init__(self, firstk, lq, lg, nf, ns):
        super(PACRR, self).__init__(name='PACRR')
        self.firstk = firstk
        self.lq = lq
        self.lg = lg
        self.nf = nf
        self.ns = ns
        
        self.conv_stack = Conv_stack(lg=self.lg, nf=self.nf)
        self.dim_wise_max_pooling = Dim_wise_max_pooling(lg=self.lg, nf=self.nf)
        self.row_wise_max_pooling = Row_wise_max_pooling(lg=self.lg, ns=self.ns, firstk=self.firstk)
        self.idf_concat = Idf_concat()
        self.recurrent_Layer = Recurrent_Layer(lq=self.lq, ns=self.ns, lg=self.lg)

    def call(self, inputs, idf):
        x = self.conv_stack(inputs)
        x = self.dim_wise_max_pooling(x)
        x = self.row_wise_max_pooling(x)
        x = self.idf_concat(x, idf)
        x = self.recurrent_Layer(x)
        return x
```

#### Evaluation

DRMM과 마찬가지로 nDCG@20값과 같은 검색어에서의 랭킹을 보겠습니다.

<img src='./img/pacrr_loss.PNG'>

<img src='./img/paccr_after_training.PNG'>

## Deep Relevance Ranking Using Enhanced Document-Query Interactions (PACRR-DRMM)

기존에 제시된 관련성 기반 모델을 조합하여 단점을 보완하고 2018년도에 SOTA를 달성한 Paper입니다. 여러가지 모델이 제안되었지만 앞서 설명한 PACRR와 DRMM를 조합한 PACRR-DRMM모델을 설명하고 구현해보겟습니다. 

### PACRR-DRMM

<img src='./img/pacrr_drmm_fig1.PNG'>
<span style="display: block; text-align: center;">Architecture of the PACRR-DRMM</span>

그림에서 볼수 있듯이 PACRR의 장점인 n-gram(CNN)기능과 DRMM의 장점인 검색어 단어별 shared MLP 시스템을 적절히 섞을것을 알수 있습니다.
바로 구현해보고 결과를 보겟습니다.

### Model Implement

```python
class PACRR_DRMM(tf.keras.Model):
    def __init__(self, firstk, lq, lg, nf, ns):
        super(PACRR_DRMM, self).__init__(name='PACRR_DRMM')
        self.firstk = firstk
        self.lq = lq
        self.lg = lg
        self.nf = nf
        self.ns = ns
        
        self.pacrr = PACRR(firstk=self.firstk, lq=self.lq, lg=self.lg, nf=self.nf, ns=self.ns)
        self.drmm = DRMM()
        
    def call(self, inputs, idf):
        x = self.pacrr(inputs, idf)
        x = self.drmm(x)
        return x
```

#### Evaluation

<img src='./img/pacrr_drmm_loss.PNG'>

<img src='./img/paccr_drmm_after_training.PNG'>